In [1]:
import configuration
import tensorflow as tf
import utils
import os
import glob

from dataset import get_datasets
from gated_stormer import Stormer

2024-05-08 07:54:51.240592: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 07:54:51.305264: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 07:54:52.295524: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-08 07:54:53.296249: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVI

1 Physical GPUs, 1 Logical GPUs


In [2]:
# print model names

print_model_table = lambda model_list: utils.print_enumerated_list(model_list, "Model")

models_names = [path.split("/")[-1] for path in glob.glob("models/*stormer*")]
models_names.sort()
print_model_table(models_names)
model_name = models_names[int(input("Choose model: "))]

Available Models:
_______________________________________________________
| Index        | Model Name                           |
|______________|______________________________________|
| 0            | gated_stormer_r2_h4_dm32_dataset=mel |
|______________|______________________________________|


In [3]:
hps = utils.load_hps(model_name)
stormer = Stormer(**hps)

In [4]:
## load the datasets
train, valid, test = get_datasets(
    **hps
)

Loading dataset version 2
Dataset loaded


2024-05-08 07:54:58.310876: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-05-08 07:54:58.315901: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


In [5]:
for example,label in train.take(1):
    stormer(example)

2024-05-08 07:55:18.043939: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
# check if the model containing directory exists
model_path = utils.get_model_path(model_name)
print("Model path:", model_path)
load_weights = os.path.exists(os.path.dirname(model_path))
if load_weights:
    stormer.load_weights(model_path)

Model path: ./models/gated_stormer_r2_h4_dm32_dataset=mel/gated_stormer_r2_h4_dm32_dataset=mel.weights.h5


In [7]:
results_filename = f'data/results/{model_name}.csv'

metrics=["accuracy"]

stormer.compile(
    optimizer=tf.keras.optimizers.AdamW(hps["learning_rate"]),
    loss="categorical_crossentropy",
    metrics=metrics,
)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

state_transformer_history = stormer.fit(
    train,
    validation_data=valid,
    epochs=hps["num_epochs"],
    callbacks=[
        model_checkpoint_callback,
        utils.MetricsLogger(
            results_filename,
        )
    ],
)

Epoch 1/10000


I0000 00:00:1715154977.742516 3325217 service.cc:145] XLA service 0x7fc6380758a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715154977.742653 3325217 service.cc:153]   StreamExecutor device (0): NVIDIA L40S, Compute Capability 8.9
2024-05-08 07:56:21.611052: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-08 07:56:32.194683: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1715155061.435221 3325217 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  35/1489 ━━━━━━━━━━━━━━━━━━━━ 45s 31ms/step - accuracy: 0.7327 - loss: 1.1740